<a href="https://colab.research.google.com/github/marianomirabal/etl-ibnorca/blob/main/notebooks/etl_ibnorca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1) Instala dependencias (solo la primera vez)
!pip install pandas openpyxl sqlalchemy

import pandas as pd
from sqlalchemy import create_engine

# 2) EXTRACT
# (antes de correrlo, sube tu Excel manualmente al entorno: panel izquierdo 📁 → subir archivo)
df_raw = pd.read_excel('/content/Clientes_faltantes1.xlsx')
print("Leídas", df_raw.shape[0], "filas y", df_raw.shape[1], "columnas.")

# 3) TRANSFORM
df = df_raw.copy()
df.columns = (
    df.columns
      .str.strip()
      .str.replace(r'\.+','_',regex=True)
      .str.replace(r'\s+','_',regex=True)
)
keywords = ['SRL','S.R.L.', 'S.R.L','SRL.','S.A','S.A.','SA','LTDA','CORPORACIÓN','COOPERATIVA','GRUPO']
df['TipoCliente'] = df['Column1_razon_social'].apply(
    lambda x: 'Empresa' if any(k in str(x).upper() for k in keywords) else 'Persona'
)
print(df[['Column1_razon_social','TipoCliente']].head())

# 4) LOAD
engine = create_engine('sqlite:///clientes_ibnorca.db')
df.to_sql('clientes_limpios', engine, if_exists='replace', index=False)
from sqlalchemy import inspect
inspector = inspect(engine)
print("Tablas en SQLite:", inspector.get_table_names())

Leídas 1660 filas y 10 columnas.
              Column1_razon_social TipoCliente
0                             CAPO     Persona
1                          SALINAS     Empresa
2   ALCIDES GUZMAN AYAVIRI AYAVIRI     Persona
3            Altavista Group S.R.L     Empresa
4                             AYMA     Persona
Tablas en SQLite: ['clientes_limpios']
